<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_2_jl" style="text-align:center; font-size:16px">III.2 Optimal Control for STIRAP</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_1_TLS.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="jl_exercise_3_1_TLS.ipynb" style="font-size:13px">III.1 Population Inversion in a Two-Level-System using Krotov's Method and GRAPE</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_2_lambda.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="jl_exercise_2_2_lambda.ipynb" style="font-size:13px">II.2 Parameter Optimization for STIRAP</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_3_chiral.ipynb">next notebook $\rightarrow$</a><br>
        <a href="jl_exercise_3_3_chiral.ipynb" style="font-size:13px">III.3 Using Krotov's method to separate chiral molecules</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_3_2_lambda.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Optimal Control for STIRAP

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid #2\vphantom{#1}\right\rangle}
\newcommand{Ketbra}[2]{\left\vert#1\vphantom{#2} \right\rangle \hspace{-0.2em} \left\langle #2\vphantom{#1}\right\vert}
\newcommand{e}[1]{\mathrm{e}^{#1}}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
\newcommand{toP}[0]{\omega_{12}}
\newcommand{toS}[0]{\omega_{23}}
\newcommand{oft}[0]{\left(t\right)}
$

In this notebook, you will learn how to optimize the STIRAP protocol for the lambda system we have studied already in [Exercise I.2](jl_exercise_1_2_lambda.ipynb) and [Exercise II.2](jl_exercise_2_2_lambda.ipynb) using the [`QuantumControl` Julia framework](https://juliaquantumcontrol.github.io/QuantumControl.jl), similarly to [Exercise III.1](jl_exercise_3_1_TLS.ipynb).

Specifically, we will use Krotov's method for the optimizations. 
There will also be a bonus exercise in the end, in which you can do
the same optimizations using GRAPE to compare it to Krotov's method.

## Setup

First, we need to load some of the libraries that we will need throughout this notebook.

In [ ]:
using QuantumControl
using Krotov

In [ ]:
const 𝕚 = 1im;

For visualization, we will use the `Plots` package

In [ ]:
using Plots

# Set up thicker default lines in plots
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

In [ ]:
# Some utilities for showing hints and solutions
include(joinpath("utils", "exercise_3_lambda.jl"));

## Model

Our model consists of a "Lambda system" as shown below and already discussed
in the respective notebooks in part 1 and part 2.
The Hamiltonian in the rotating frame, which we already derived in part 1, is
given as:

\begin{align*}
  H = \begin{pmatrix}
        0                    & \frac{1}{2}\Omega_{P}^{*}(t) & 0 \\
    \frac{1}{2}\Omega_{P}(t) & \Delta_P                     & \frac{1}{2}\Omega_{S}^{*}(t)\\
        0                    & \frac{1}{2}\Omega_{S}(t)     & \Delta_P-\Delta_S
  \end{pmatrix}
\end{align*}

we use the same definitions as in [Exercise I.2](jl_exercise_1_2_lambda.ipynb), i.e.

$\Delta_{\mathrm{P}} = \left(\omega_2 - \omega_1\right) - \omega_{\mathrm{P}}$ and
$\Delta_{\mathrm{S}} = \left(\omega_2 - \omega_3 \right) - \omega_{\mathrm{S}}$.

Note that the Rabi frequencies are complex, i.e.

$\Omega_{\mathrm{P}} = \Omega^{(1)}_{\mathrm{P}} + i\Omega^{(2)}_{\mathrm{P}}$ and
$\Omega_{\mathrm{S}} = \Omega^{(1)}_{\mathrm{S}} + i\Omega^{(2)}_{\mathrm{S}}$.

In the following, we will optimize the real and imaginary part of
$\Omega_{\mathrm{S}}$ and $\Omega_{\mathrm{P}}$ independently.

<center><img src="../figures/lambda_system_levels.png" alt="Lambda system considered in this notebook" width="500"></center>


First, we set up the Hamiltonian

In [ ]:
using QuantumControl: hamiltonian

In [ ]:
ω₁ = 0.0;
ω₂ = 10.0;
ω₃ = 5.0;
ω_P = 9.5;
ω_S = 4.5;

In [ ]:
Δ_P = (ω₂ - ω₁) - ω_P;
Δ_S = (ω₂ - ω₃) - ω_S;

We begin with the drift Hamltonian:

In [ ]:
using LinearAlgebra: Diagonal
H0 = Array(Diagonal(ComplexF64[0, Δ_P, Δ_P - Δ_S]))

Next, we define the "pump" Hamiltonian:

In [ ]:
H1P_re = 0.5 * ComplexF64[
    0  1  0
    1  0  0
    0  0  0
]

In [ ]:
H1P_im =  0.5 * ComplexF64[
    0  𝕚  0
   -𝕚  0  0
    0  0  0
]

And lastly, the "Stokes" Hamiltonian:

In [ ]:
H1S_re = 0.5 * ComplexF64[
    0  0  0
    0  0  1
    0  1  0
]

In [ ]:
H1S_im =  0.5 * ComplexF64[
    0  0  0
    0  0  𝕚
    0 -𝕚  0
]

We combine these into the full Hamiltonian using the
[`QuantumControl.hamiltonian`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_propagators/#QuantumPropagators.Generators.hamiltonian)
function. However, we still need to define suitable guess controls.

## Guess pulses

We choose an initial guess consisting of two low-intensity [Blackman
pulses](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_propagators/#QuantumPropagators.Shapes.blackman).

In [ ]:
using QuantumControl.Shapes: blackman

Initially, these are chosen to be real-valued. That is, the guess for the control Hamiltonians governing the imaginary part of the Rabi frequencies will be zero. Thus, the total Hamiltonian is


In [ ]:
H = hamiltonian(
    H0,
    (H1P_re, t -> blackman(t, 1.0, 5.0)),
    (H1P_im, t -> 0.0),
    (H1S_re, t -> blackman(t, 0.0, 4.0)),
    (H1S_im, t -> 0.0)
);

These pulses are defined on the time grid

In [ ]:
tlist = collect(range(0, 5; length=501));

They look as follows:

In [ ]:
using QuantumControl.Controls: get_controls

In [ ]:
function plot_pulses(H, tlist)
    axs = []
    controls = get_controls(H)
    @assert length(controls) == 4
    titles = ["Re[Ωₚ]", "Im[Ωₚ]", "Re[Ωₛ]", "Im[Ωₛ]"]
    for (i, control) in  enumerate(controls)
        ax = plot(
            tlist, control;
            label="", xlabel="time", ylabel="pulse amplitude",
            title=titles[i]
        )
        push!(axs, ax)
    end
    plot(axs...; layout=4)
end

In [ ]:
plot_pulses(H, tlist)

After having set up everything, let's see how good our guess is!
To that end, we simulate the dynamics of the initial state

In [ ]:
ket1 = ComplexF64[1, 0, 0]

In [ ]:
using QuantumPropagators: propagate
using QuantumPropagators: Cheby

In [ ]:
guess_dynamics = propagate(ket1, H, tlist; method=Cheby, storage=true)

In [ ]:
plot(
    tlist, abs2.(guess_dynamics)';
    label=["⟨1|Ψ|1⟩" "⟨2|Ψ|2⟩" "⟨3|Ψ|3⟩"],
    xlabel="time", ylabel="population",
)

## Problem 1: Optimization trajectory

The `QuantumControl` package defines control objectives on top of a set of
"trajectories". These hold information about the quantum states that span the
optimization, their dynamics, and (optionally) the "target" of those
dynamics.

Define the
[Trajectory](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_control_base/#QuantumControlBase.Trajectory) that encodes the goal for transferring the state $\ket{1}$ to $\ket{3}$.

In [ ]:
ket3 = ComplexF64[0, 0, 1]

In [ ]:
trajectory = Trajectory(#= .... =#)

In [ ]:
# problem_1.hint

In [ ]:
# problem_1.solution

## Problem 2: Specifying pulse options

Now that our Hamiltonian is completely set up and the trajectory to
be used in the optimization is defined, we need to specify some parameters
for Krotov's method, specifically, a dictionary of "pulse options".

First, we define a "pulse shape". This must be a function in the range [0, 1]
which switch on from zero at the beginning of the time grid, and goes back down
to zero again at the end of the time grid. It furthermore scales the pulse update in
Krotov's method at each point in time, and thus ensures that the physical
boundary conditions of the control fields are maintained (i.e. they must remain
zero at the beginning and end of the time grid). We begin with the following choice

In [ ]:
using QuantumControl.Shapes: flattop

In [ ]:
S(t) = flattop(t; T=tlist[end], t_rise=0.0001, func=:sinsq)

**a)** Experiment with the `t_rise` parameter and plot the update shape with
the following cell. Choose a reasonable value.

Feel free to return to this point later on to experiment with the pulse shape and its impact on the
optimization result.

In [ ]:
plot(tlist, S; xlabel="time", ylabel="update shape", label="", marker=true)

In [ ]:
# problem_2a.solution

In addition to the "update shape", the pulse options must also include a parameter $\lambda_a$ that determines the overall magnitude of the pulse
update in each iteration. In Krotov's update equation, the update at each
point in time is scaled with an overall factor of $S(t)/\lambda_a$, so larger
values of update suppress the update, while smaller values increase the
update, but tend to make the update more "spiky" and may lead to unphysical
results. Here, we start with a relatively safe (i.e. large) value of $\lambda_a =
100$.

**b)** After setting up the optimization in problem 3, return and choose a
better value $\lambda_a$ that yields faster convergence but does not produce
unphysical pulses.

In [ ]:
λₐ = 100.0
pulse_options = IdDict(
    ϵ => Dict(:lambda_a => λₐ, :update_shape => S,)
    for ϵ in get_controls(H)
)

In [ ]:
# problem_2b.hint

In [ ]:
# problem_2b.solution

## Problem 3: Optimization with Krotov's method

Finally, we can put all of the above together into a
[`QuantumControl.ControlProblem`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_control_base/#QuantumControlBase.ControlProblem)

Fill in the following missing values, which are indicated by `#= … =#`.
Proceeds as follows:

**a)** Recall the API of the
[`ControlProblem`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_control_base/#QuantumControlBase.ControlProblem),
respectively the
[`QuantumControl.optimize`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/methods/#QuantumControlBase.optimize-Tuple{ControlProblem,%20Val{:Krotov}}-methods)
function.

**b)** Which functional do we need here? Choose from the functionals
implemented in
[`QuantumControl.Functionals`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_control_reference/#QuantumControlFunctionalsLocalAPI).

In [ ]:
using QuantumControl.Functionals

**c)** What do the values for the `check_convergence` and `iter_stop`
argument mean? Make a reasonable choice here.

**d)** Adjust the relevant parameters to obtain a better convergence.

Ensure that the changes you apply lead to reasonable results. If one wants to optimize for an experiment, the optimized pulses need to be physical such that they can be implemented in practice.

In [ ]:
problem = ControlProblem(
    #= (a) =#,  # positional argument 1
    #= (a) =#;  # positional argument 2
    #= (a) =#,  # Krotov-specific keyword argument(s)
    J_T=#= (b) =#,
    iter_stop=#= (c) =#,
    check_convergence=res -> begin
        #= (c) =#,
    end,
    prop_method=Cheby,
);

In [ ]:
# problem_3b.hint

In [ ]:
# problem_3c.hint

In [ ]:
# problem_3d.hint

In [ ]:
# problem_3.solution

In [ ]:
oct_result = optimize(problem; method=Krotov)

## Problem 4: Analyzing the results

Finally, we can have a look at our solution.

**a)** Construct the Hamiltonian containing the optimized controls. This can
be achieved using the
[`substitute`](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_propagators/#QuantumPropagators.Controls.substitute)
function

In [ ]:
using QuantumControl.Controls: substitute

to replace the guess controls in the original Hamiltonian (`get_controls(H)`)
with the optimized controls stored in the `optimized_controls` attribute of
`oct_result`.

In [ ]:
H_opt = substitute(
    #= =#
);

In [ ]:
# problem_4a.hint

In [ ]:
# problem_4a.solution

**b)** Simulate the dynamics of the $\ket{1}$ state under `H_opt` such that
we can later plot the population (just like we did earlier with `H`
containing the guess pulses)

In [ ]:
opt_dynamics = propagate( #= =# )

In [ ]:
# problem_4b.hint

In [ ]:
# problem_4b.solution

After simulating the optimized dynamics we can plot them via

In [ ]:
plot(
    tlist, abs2.(opt_dynamics)';
    label=["⟨1|Ψ|1⟩" "⟨2|Ψ|2⟩" "⟨3|Ψ|3⟩"],
    xlabel="time", ylabel="population",
)

We can also visualize the optimized pulses themselves. We can convert the
separate pulses for the real and imaginary part of $\Omega_S$ and
$\Omega_P$ into an absolute value and complex phase for the plot:

In [ ]:
function plot_pulses_abs_phase(H, tlist)
    axs = []
    controls = get_controls(H)
    @assert length(controls) == 4
    ΩP_re, ΩP_im, ΩS_re, ΩS_im = controls
    ΩP = ΩP_re + 𝕚 * ΩP_im
    ΩS = ΩS_re + 𝕚 * ΩS_im
    data = [abs.(ΩP), angle.(ΩP) ./ π, abs.(ΩS), angle.(ΩS) ./ π,]
    titles = ["|Ωₚ|", "arg(Ωₚ)", "|Ωₛ|", "arg(Ωₛ)"]
    ylabels = ["pulse amplitude", "phase (π)", "pulse amplitude", "phase (π)"]
    for (i, y) in  enumerate(data)
        ax = plot(
            tlist, y;
            label="", xlabel="time",
            ylabel=ylabels[i], title=titles[i]
        )
        push!(axs, ax)
    end
    plot(axs...; layout=4)
end

In [ ]:
plot_pulses_abs_phase(H_opt, tlist)

# Bonus: Optimization with GRAPE

In [ ]:
using GRAPE

**a)** Try rerunning the optimization with GRAPE and look at the resulting optimized pulses and dynamics

In [ ]:
oct_result_grape = optimize( #= =# )

In [ ]:
H_opt_grape = substitute(
    H,
    #= =#
);

In [ ]:
plot_pulses_abs_phase(H_opt_grape, tlist)

In [ ]:
opt_dynamics_grape = propagate( #= =# )

In [ ]:
plot(
    tlist, abs2.(opt_dynamics_grape)';
    label=["⟨1|Ψ|1⟩" "⟨2|Ψ|2⟩" "⟨3|Ψ|3⟩"],
    xlabel="time", ylabel="population",
)

Do you observe any potential issues?

In [ ]:
# bonus_a.hint

In [ ]:
# bonus_a.solution

**b)** Looking at the previous exercise, what would you do to alleviate these
issues? Here is a hint:

In [ ]:
using QuantumControl.Amplitudes: ShapedAmplitude

You can try to implement your solution, although for analyzing the results
you may have to deal with some subtle issues related to time discretization
in the `QuantumControl` framework. You would have to set up a modified
Hamiltonian, and recreate the `Trajectory` and `ControlProblem` with that
modified Hamiltonian

In [ ]:
using QuantumControl.Controls: discretize, discretize_on_midpoints

In [ ]:
# bonus_b.hint

In [ ]:
# bonus_b.solution

## Next steps

If you are interested in more examples of gradient-based optimization, have a look at [Exercise III.3](jl_exercise_3_3_chiral.ipynb) which covers the topic of three-wave-mixing to distinguish between enantiomers of a chiral molecule. Alternatively, if you are more interested in gate optimization for quantum information applications, we recommend [Exercise III.4](jl_exercise_3_4_gate.ipynb).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_2_jl)
<!-- Autofooter end -->